In [4]:
from pymongo import MongoClient
from IPython.display import display, Markdown

# Connect to MongoDB
client = MongoClient("mongodb://localhost:28910/")
db = client["wifi_data_db"]
collection = db["wifi_client_data"]

# More robust query that checks if data exists and is an array
pipeline = [
    {
        "$match": {
            "data": {"$exists": True, "$type": "array"}
        }
    },
    {
        "$addFields": {
            "funNetworkCount": {
                "$size": {
                    "$filter": {
                        "input": "$data",
                        "as": "item",
                        "cond": { 
                            "$and": [
                                {"$ifNull": ["$$item.SSID", False]},
                                {"$eq": ["$$item.SSID", "fun_network"]}
                            ]
                        }
                    }
                }
            }
        }
    },
    {
        "$match": {
            "funNetworkCount": 3
        }
    },
    {
        "$count": "totalDocuments"
    }
]

try:
    result = list(collection.aggregate(pipeline))
    count = result[0]["totalDocuments"] if result else 0
    display(Markdown(f"**Number of entries with exactly 3 'fun_network' occurrences:** {count}"))
except Exception as e:
    display(Markdown(f"**Error:** {str(e)}"))

**Number of entries with exactly 3 'fun_network' occurrences:** 23839

In [5]:
import pandas as pd
from IPython.display import display, Markdown

# Set pandas display options to show all rows and columns
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', None)

pipeline = [
    {
        "$match": {
            "metadata.pico_ip": {"$exists": True},
            "metadata.button_id": {"$exists": True}
        }
    },
    {
        "$addFields": {
            "y_coordinate": {
                "$toInt": {
                    "$arrayElemAt": [
                        {"$split": ["$metadata.pico_ip", "."]},
                        3
                    ]
                }
            }
        }
    },
    {
        "$group": {
            "_id": {
                "x": "$metadata.button_id",
                "y": "$y_coordinate"
            },
            "count": { "$sum": 1 }
        }
    },
    {
        "$sort": { "_id.x": 1, "_id.y": 1 }
    }
]

try:
    result = list(collection.aggregate(pipeline))
    df = pd.DataFrame([{
        "X (button_id)": item["_id"]["x"],
        "Y (pico_ip last digit)": item["_id"]["y"],
        "Count": item["count"]
    } for item in result])
    
    display(Markdown("**Number of points per X,Y coordinate:**"))
    display(df)
    
    # Reset pandas options to default after display
    pd.reset_option('display.max_rows')
    pd.reset_option('display.max_columns')
    pd.reset_option('display.width')
    pd.reset_option('display.max_colwidth')
except Exception as e:
    display(Markdown(f"**Error:** {str(e)}"))

**Number of points per X,Y coordinate:**

,X (button_id),Y (pico_ip last digit),Count
0,1,30,681
1,1,31,681
2,1,32,681
3,1,33,681
4,1,34,681
5,1,35,681
6,1,36,681
7,1,37,681
8,1,38,681
9,1,39,681


In [6]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from pymongo import MongoClient
from datetime import datetime
import plotly.io as pio
pio.renderers.default = 'browser'  # Opens plot in default browser

# Set time interval (UNIX timestamps)
start_time = datetime(2025, 5, 13, 20, 10).timestamp()
end_time = datetime(2025, 5, 13, 21, 42).timestamp()

# Define the custom order for pico IPs
custom_pico_order = [31, 32, 33, 34, 35, 36, 37, 38, 39, 30]
pico_mapping = {val: idx for idx, val in enumerate(custom_pico_order)}

# MongoDB query to get unique BSSIDs (same as before)
bssid_pipeline = [
    {
        "$match": {
            "timestamp": {"$gte": start_time, "$lte": end_time},
            "data.SSID": "fun_network"
        }
    },
    {"$unwind": "$data"},
    {"$match": {"data.SSID": "fun_network"}},
    {"$group": {"_id": "$data.BSSID"}},
    {"$sort": {"_id": 1}}
]

unique_bssids = [x["_id"] for x in collection.aggregate(bssid_pipeline)]

if not unique_bssids:
    print("No BSSIDs found in the specified time range")
else:
    print(f"Found {len(unique_bssids)} fun_network BSSIDs between "
          f"{datetime.fromtimestamp(start_time)} and {datetime.fromtimestamp(end_time)}")

    # Create subplot for each BSSID
    fig = make_subplots(
        rows=len(unique_bssids), cols=1,
        specs=[[{'type': 'scatter3d'}] for _ in unique_bssids],
        subplot_titles=[f"BSSID: {bssid}" for bssid in unique_bssids],
        vertical_spacing=0.05
    )

    for i, bssid in enumerate(unique_bssids, 1):
        # Query for this specific BSSID (same as before)
        pipeline = [
            {
                "$match": {
                    "timestamp": {"$gte": start_time, "$lte": end_time},
                    "data": {
                        "$elemMatch": {
                            "SSID": "fun_network",
                            "BSSID": bssid
                        }
                    }
                }
            },
            {
                "$addFields": {
                    "y_coordinate": {
                        "$toInt": {
                            "$arrayElemAt": [
                                {"$split": ["$metadata.pico_ip", "."]}, 
                                3
                            ]
                        }
                    },
                    "filtered_data": {
                        "$filter": {
                            "input": "$data",
                            "as": "item",
                            "cond": {
                                "$and": [
                                    {"$eq": ["$$item.SSID", "fun_network"]},
                                    {"$eq": ["$$item.BSSID", bssid]}
                                ]
                            }
                        }
                    }
                }
            },
            {"$unwind": "$filtered_data"},
            {
                "$project": {
                    "x": "$metadata.button_id",
                    "y": "$y_coordinate",
                    "z": "$filtered_data.RSSI",
                    "timestamp": 1,
                    "_id": 0
                }
            }
        ]
        
        data = list(collection.aggregate(pipeline))
        
        if not data:
            print(f"No data found for BSSID: {bssid}")
            continue
            
        df = pd.DataFrame(data)
        df['y_mapped'] = df['y'].map(pico_mapping)
        df = df.sort_values('x')  # Sort by button_id

        # Add 3D scatter plot for this BSSID
        fig.add_trace(
            go.Scatter3d(
                x=df['x'],
                y=df['y_mapped'],
                z=df['z'],
                mode='markers',
                marker=dict(
                    size=5,
                    color=df['z'],
                    colorscale='Viridis',
                    opacity=0.8,
                    colorbar=dict(title='RSSI (dBm)')
                ),
                name=bssid,
                text=[f"Button: {x}<br>Pico: {y}<br>RSSI: {z}dBm" 
                     for x, y, z in zip(df['x'], df['y'], df['z'])]
            ),
            row=i, col=1
        )

        # Update subplot layout
        fig.update_scenes(
            xaxis_title='Button ID (1-10)',
            yaxis_title='Pico IP',
            zaxis_title='RSSI (dBm)',
            yaxis=dict(
                tickvals=list(range(len(custom_pico_order))),
                ticktext=[str(x) for x in custom_pico_order]
            ),
            xaxis=dict(
                tickvals=list(range(1, 11)),
                ticktext=[str(x) for x in range(1, 11)]
            ),
            row=i, col=1
        )

    # Update overall layout
    fig.update_layout(
        title_text=f"Wi-Fi Signal Strength Analysis<br>"
                  f"Time Range: {datetime.fromtimestamp(start_time)} to {datetime.fromtimestamp(end_time)}",
        height=300 * len(unique_bssids),
        margin=dict(l=0, r=0, b=0, t=100),
        scene_camera=dict(eye=dict(x=1.5, y=1.5, z=0.8))  # Default camera angle
    )

    # Add dropdown menu to change camera angles
    buttons = [
        dict(
            label="Default View",
            method="relayout",
            args=[{"scene.camera": {"eye": {"x": 1.5, "y": 1.5, "z": 0.8}}}]
        ),
        dict(
            label="Top View",
            method="relayout",
            args=[{"scene.camera": {"eye": {"x": 0, "y": 0, "z": 2.5}}}]
        ),
        dict(
            label="Side View",
            method="relayout",
            args=[{"scene.camera": {"eye": {"x": 2.5, "y": 0, "z": 0}}}]
        )
    ]

    fig.update_layout(
        updatemenus=[{
            "type": "dropdown",
            "buttons": buttons,
            "x": 1.0,
            "xanchor": "right",
            "y": 1.15,
            "yanchor": "top"
        }]
    )

    
    fig.show()

Found 3 fun_network BSSIDs between 2025-05-13 20:10:00 and 2025-05-13 21:42:00
